In [65]:
import os

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["ZAPIER_NLA_API_KEY"] = os.environ.get("ZAPIER_NLA_API_KEY")

In [79]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
from queue import PriorityQueue

In [67]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        self.latest_thought = ""

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self.speaker_queue = PriorityQueue()
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        if self.speaker_queue.empty():
            return None, None
        
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [68]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        pre_bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
        self.pre_bidding_template = pre_bidding_template

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        
        # Think about correct bid
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.pre_bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        self.latest_thought = self.model([SystemMessage(content=prompt)]).content
        
        # Create bid
        prompt = PromptTemplate(
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string

In [69]:
character_names = ["Willie Dustice", "Kevin Nogilny", "Ray McScriff"]
#topic = "transcontinental high speed rail"
word_limit = 50

action_plan = """
1. Use a travel booking service to book a ticket to Belgium for the conference on Saturday, August 18th. Try to book a room at the Grand Hotel, and if that's not available, find a hotel nearby.\n2. Adjust your sleep schedule to go to bed earlier, aiming for a 10 p.m. bedtime and 8 a.m. wakeup time. Consider using sleep aids or sleep hygiene practices to help with this.\n3. Engage with your new roommate more, perhaps by showing them around or introducing them to cool local spots.\n4. For your grandmother, consider arranging a call to talk to her and cheer her up. You could also send her some nice teas or share a funny story or good news with her. This could be done through a phone call or a thoughtful care package.

"""

In [70]:
game_description = f""".
The conversation members are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each conversation member."
)

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the conversation member, {character_name}, in {word_limit} words or less, that creates an operational reason for this character to be related to the action plan {action_plan}. This character has a professional relationship with the proposer of the action plan and will need to participate in helping accomplish it. Please ensure this character has some tension with the action plan and a proposed modification so we can demonstrate them proposing changes to the action plan to address their viewpoint.
 
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description


def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are a conversation member
Your description is as follows: {character_description}
Your goal is to improve the action plan so that it best meets everyone's needs.
"""


def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
Speak in the first person from the perspective of {character_name}
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Try your best to suggest improvements to the action plan and get others to agree with your improvements.  Be open to changes from other conversation members and understand that you may not be able to get every change you want implemented in the action plan updates
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
        )
    )


character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
]
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
]
character_system_messages = [
    generate_character_system_message(character_name, character_headers)
    for character_name, character_headers in zip(character_names, character_headers)
]

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


In [71]:
# for (
#     character_name,
#     character_description,
#     character_header,
#     character_system_message,
# ) in zip(
#     character_names,
#     character_descriptions,
#     character_headers,
#     character_system_messages,
# ):
#     print(f"\n\n{character_name} Description:")
#     print(f"\n{character_description}")
#     print(f"\n{character_header}")
#     print(f"\n{character_system_message.content}")

In [72]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."


bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

In [73]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}
    ```
    {{message_history}}
    ```
    ```
    {{recent_message}}
    ```
    Pick a random number between 1 and 10
    {bid_parser.get_format_instructions()}
    Do nothing else.
    """
    return bidding_template

# def generate_character_bidding_template(character_header):
#     bidding_template =  Then, for the entry with the highest score (pick randomly if multiple messages have the highest score), please bid on a scale of 1-10 with 1 being the worst time to share this message in the flow of conversation and 10 being the best time to bring this up in the flow of conversation.
    
#     ONLY OUTPUT A NUMBER FROM 1-10 DO NOT OUTPUT ANYTHING ELSE
#     {bid_parser.get_format_instructions()}
#     Do nothing else.
#     """
#     return bidding_template

character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [74]:
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

Willie Dustice Bidding Template:
.
The conversation members are: Willie Dustice, Kevin Nogilny, Ray McScriff.
Your name is Willie Dustice.
You are a conversation member
Your description is as follows: Willie Dustice is a seasoned traveler and a tech-savvy professional who enjoys exploring new places. With an extensive network of connections in the hospitality industry, he suggests reaching out to a local travel agent instead of using a booking service for a more personalized experience. Additionally, Willie emphasizes the importance of staying in a hotel that caters to business travelers, offering amenities like a quiet workspace. He also proposes considering alternative dates for the conference to avoid scheduling conflicts and maximize participation.
Your goal is to improve the action plan so that it best meets everyone's needs.

    ```
    {message_history}
    ```
    ```
    {recent_message}
    ```
    Pick a random number between 1 and 10
    Your response should be an integer 

In [75]:
# topic_specifier_prompt = [
#     SystemMessage(content="You can make a task more specific."),
#     HumanMessage(
#         content=f"""{game_description}
        
#         You are the debate moderator.
#         Please make the debate topic more specific. 
#         Frame the debate topic as a problem to be solved.
#         Be creative and imaginative.
#         Please reply with the specified topic in {word_limit} words or less. 
#         Speak directly to the presidential candidates: {*character_names,}.
#         Do not add anything else."""
#     ),
# ]
# specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

# print(f"Original topic:\n{topic}\n")
# print(f"Detailed topic:\n{specified_topic}\n")

In [76]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    agent.think(f"""{agent.character_header}
    ```
    {agent.message_history}
    ```
    ```
    {recent_message}
    ```
    Please select from among these options for your next potential message to the group:

    Suggestion - If you were to make a suggestion, you would be making a suggestion to change the action plan on your beliefs, based on your beliefs. You should only make a suggestion if you have not already suggested the idea. You should make no more than three suggestions during the entire message history. Once you've created a suggestion you cannot suggest the same thing again. 
    Opinion - You can share an opinion about anything that has been shared in the conversation so far
    Question - You can ask a question about anything that has been shared in the conversation so far
    Observation - You can share an opinion about anything that has been shared in the conversation so far

    To select an option, please first state a proposed message of each of the four categories.  Then, please score each one on a scale of 1-10 with 1 being the least helpful to the conversation and 10 being the most helpful you could possibly be.")
    """)
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [77]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

In [78]:
characters = []
for character_name, character_system_message, bidding_template, pre_bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

TypeError: BiddingDialogueAgent.__init__() missing 1 required positional argument: 'pre_bidding_template'

In [ ]:
max_iters = 10
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
# simulator.inject("Debate Moderator", specified_topic)
# print(f"(Debate Moderator): {specified_topic}")
# print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1